Import modules requires for running

In [2]:
import geopandas as gpd
from synthetic_offsets.dem import deform_dem
from synthetic_offsets.fault import Fault

Supply names of DEM, fault trace shapefile (both NZTM) and fault source parameters

In [3]:
dem = "BU23.tif"
trace = "simplified_kakapo.geojson"

fault = Fault(trace=trace, dip=80., boundary=dem, slip=-10., rake=180., dip_direction=340.)

Initializing Fault object...
Dip direction (azimuth is 340.00)
Change by overriding or using flip_dip_direction option...


Create deformed DEM (may throw up a warning but should still run)

In [4]:
hw, fw = fault.footwall_and_hangingwall_polygons(fault.clipped_trace)
e, n, u = fault.slip_and_rake
hw_shifted, clipped_fw = fault.moved_footwall_hangingwall_polygons(fault.clipped_trace, x_shift=e, y_shift=n)
holes = gpd.GeoSeries(hw.difference(hw_shifted), crs=2193)
deform_dem(dem, "kakapo_scarp.tif", hw_shifted, clipped_fw, holes, e, n, u)

[[[477.005   476.983   476.95898 ... 529.004   529.138   529.242  ]
  [476.955   476.901   476.829   ... 529.52    529.68604 529.784  ]
  [476.769   476.669   476.619   ... 529.976   530.096   530.13   ]
  ...
  [783.26996 782.566   781.84595 ... 618.326   618.62    619.052  ]
  [782.771   782.181   781.521   ... 618.823   619.157   619.547  ]
  [782.401   781.729   781.171   ... 619.509   619.693   620.337  ]]]
[[[-9999.      -9999.      -9999.      ... -9999.      -9999.
   -9999.     ]
  [-9999.      -9999.      -9999.      ... -9999.        659.158
     658.88   ]
  [-9999.      -9999.      -9999.      ...   659.362     659.04004
     658.70404]
  ...
  [  783.26996   782.566     781.84595 ...   618.326     618.62
     619.052  ]
  [  782.771     782.181     781.521   ...   618.823     619.157
     619.547  ]
  [  782.401     781.729     781.171   ...   619.509     619.693
     620.337  ]]]
[[[  586.3168    586.74976   587.3912  ... -9999.      -9999.
   -9999.     ]
  [  586.94293

In [10]:
len(list(holes.geometry[0].geoms))

8

In [4]:
import geopandas as gpd
across_lines = gpd.read_file("across.geojson")
parallel_south = gpd.read_file("parallel_south.geojson")
parallel_north = gpd.read_file("parallel_north.geojson")

In [ ]:
from synthetic_offsets.io.array_operations import profile_tiff_along_linestring

In [4]:
u

np.float64(-1.2060416625018979e-15)

In [ ]:
profile_tiff_along_linestring("kakapo_scarp.tif", across_lines.geometry.iloc[0], spacing=1)

In [20]:
from synthetic_offsets.io.array_operations import read_tiff, read_grid



In [27]:
line = across_lines.geometry.iloc[0]
x, y, z = read_tiff("kakapo_scarp.tif", window=line.bounds, make_y_ascending=True, nan_threshold=1.e4)

In [28]:
z

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]],
      shape=(260, 240), dtype=float32)

In [18]:
import numpy as np
distances = np.arange(0., line.length, 1.)
sample_points = [line.interpolate(d) for d in distances]
sample_coords = [(pt.x, pt.y) for pt in sample_points]
for point in sample_points:
    print(point.x, point.y)
    nearest_xi = np.abs(x - point.x).argmin()
    nearest_yi = np.abs(y - point.y).argmin()
    nearest_z = z[nearest_yi, nearest_xi]
    print(f"Nearest DEM value: {nearest_z}")

1553765.8655436204 5280117.543509217
Nearest DEM value: nan
1553765.3108434242 5280116.711458922
Nearest DEM value: nan
1553764.756143228 5280115.879408629
Nearest DEM value: nan
1553764.2014430317 5280115.047358334
Nearest DEM value: nan
1553763.6467428356 5280114.215308039
Nearest DEM value: nan
1553763.0920426394 5280113.383257745
Nearest DEM value: nan
1553762.5373424431 5280112.551207451
Nearest DEM value: nan
1553761.9826422469 5280111.719157157
Nearest DEM value: nan
1553761.4279420506 5280110.887106862
Nearest DEM value: nan
1553760.8732418544 5280110.055056568
Nearest DEM value: nan
1553760.318541658 5280109.223006274
Nearest DEM value: nan
1553759.763841462 5280108.390955979
Nearest DEM value: nan
1553759.2091412658 5280107.558905685
Nearest DEM value: nan
1553758.6544410696 5280106.726855391
Nearest DEM value: nan
1553758.0997408733 5280105.894805096
Nearest DEM value: nan
1553757.545040677 5280105.0627548015
Nearest DEM value: nan
1553756.9903404808 5280104.230704508
Neares

In [23]:
read_grid("BU23.tif")

(array([1549218.00000001, 1549219.00000001, 1549220.00000001, ...,
        1561513.00000001, 1561514.00000001, 1561515.00000001],
       shape=(12298,)),
 array([5283735.00000001, 5283734.00000001, 5283733.00000001, ...,
        5277577.00000001, 5277576.00000001, 5277575.00000001],
       shape=(6161,)),
 array([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],
       shape=(6161, 12298), dtype=float32))

In [29]:
import rasterio
raster = rasterio.open("kakapo_scarp.tif")

In [30]:
raster.bounds

BoundingBox(left=1549218.0000000063, bottom=5277574.000000007, right=1561516.0000000063, top=5282549.000000007)

In [31]:
raster.read(1)

array([[-89404.69 , -89404.25 , -89403.61 , ..., -89331.84 , -89331.84 ,
        -89332.12 ],
       [-89404.055, -89403.62 , -89403.07 , ..., -89331.64 , -89331.84 ,
        -89332.12 ],
       [-89403.49 , -89402.945, -89402.44 , ..., -89331.64 , -89331.96 ,
        -89332.3  ],
       ...,
       [-89207.734, -89208.44 , -89209.16 , ..., -89372.67 , -89372.38 ,
        -89371.945],
       [-89208.23 , -89208.82 , -89209.48 , ..., -89372.18 , -89371.84 ,
        -89371.45 ],
       [-89208.6  , -89209.27 , -89209.83 , ..., -89371.49 , -89371.305,
        -89370.664]], shape=(4975, 12298), dtype=float32)